# 02 - Fine-tuning del Modelo

Fine-tuning de DistilBERT en la tarea MRPC.

In [1]:
!pip install transformers datasets

In [2]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# Cargar dataset y tokenizer
dataset = load_dataset('glue', 'mrpc')
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

/home/cleber/Escritorio/PC3_NLP/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def preprocess(batch):
    return tokenizer(batch['sentence1'], batch['sentence2'], truncation=True, padding='max_length')
dataset = dataset.map(preprocess, batched=True)
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map: 100%|██████████| 408/408 [00:00<00:00, 5688.65 examples/s]


In [4]:
# Configuración de entrenamiento
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
args = TrainingArguments(
    output_dir='../outputs/models/finetuned',
    eval_steps=500,  # Evaluar cada 500 pasos
    save_steps=500,  # Guardar el modelo cada 500 pasos
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Definir Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    tokenizer=tokenizer
)

# Entrenar y evaluar
trainer.train()
metrics = trainer.evaluate()
print(metrics)

/tmp/ipykernel_1802952/1500214221.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.436500


{'eval_loss': 0.3987159729003906, 'eval_runtime': 72.3572, 'eval_samples_per_second': 5.639, 'eval_steps_per_second': 0.18, 'epoch': 3.0}


In [6]:
# Guardar modelo fine-tuneado
trainer.save_model('../outputs/models/finetuned')